In [1]:
# Import libraries

import pandas as pd

In [2]:
# Load file

raw_df = pd.read_json('../data/result.json')
df = pd.json_normalize(raw_df['metadata']).drop(['white.pieces', 'black.pieces'], axis=1).reset_index()
df.head(5)

,index,opening,eco,result,date,event,white.name,white.elo,white.moves,white.distance,white.perMove,black.name,black.elo,black.moves,black.distance,black.perMove
0,0,Sicilian Defense: Venice Attack,B56,1/2-1/2,2024.04.04,FIDE Candidates 2024,"Caruana, Fabiano",2803,41,85.5,2.085,"Nakamura, Hikaru",2789,41,93.0,2.268
1,1,Queen's Gambit Declined,D53,1/2-1/2,2024.04.04,FIDE Candidates 2024,"Abasov, Nijat",2632,34,58.0,1.706,"Nepomniachtchi, Ian",2758,33,64.0,1.939
2,2,"Ruy Lopez: Open, Classical Defense",C83,1/2-1/2,2024.04.04,FIDE Candidates 2024,"Firouzja, Alireza",2760,39,60.0,1.538,Praggnanandhaa R,2747,38,73.0,1.921
3,3,Tarrasch Defense: Symmetrical Variation,D32,1/2-1/2,2024.04.04,FIDE Candidates 2024,Gukesh D,2743,21,28.0,1.333,"Vidit, Santosh Gujrathi",2727,21,35.0,1.667
4,4,Queen's Gambit Declined,D30,0-1,2024.04.04,FIDE Candidates 2024,Praggnanandhaa R,2747,33,57.0,1.727,Gukesh D,2743,33,56.5,1.712


In [3]:
# white_result_mapping = {'1-0': 1, '0-1': -1, '1/2-1/2': 0}
# white_df = pd.DataFrame({
#   'index': df['index'],
#   'opening': df['opening'],
#   'eco': df['eco'],
#   'date': df['date'],
#   'event': df['event'],
#   'playerName': df['white.name'],
#   'opponentName': df['black.name'],
#   'color': 1,
#   'result': df['result'].map(white_result_mapping),
#   'moves': df['white.moves'],
#   'ratingDifference': df['white.elo'].astype(int) - df['black.elo'].astype(int),
#   'distance': df['white.distance'],
#   'distanceDifference': df['white.distance'].astype(float) - df['black.distance'].astype(float),
#   'distancePerMove': df['white.perMove'],
#   'distancePerMoveDifference': df['white.perMove'].astype(float) - df['black.perMove'].astype(float),
# })

In [4]:
# Add transformation functions

def split_by_color(df, is_white):
  color = 1 if is_white else 0
  flip_sign = (-1)**color

  player_name = 'white.name' if is_white else 'black.name'
  opponent_name = 'black.name' if is_white else 'white.name'
  rating = 'white.elo' if is_white else 'black.elo'
  distance = 'white.distance' if is_white else 'black.distance'
  effort = 'white.perMove' if is_white else 'black.perMove'

  result_mapping = { '1-0': -flip_sign, '0-1': flip_sign, '1/2-1/2': 0 }

  result_df = pd.DataFrame({
    'index': df['index'],
    'opening': df['opening'],
    'eco': df['eco'],
    'date': df['date'],
    'event': df['event'],
    'player_name': df[player_name],
    'opponent_name': df[opponent_name],
    'color': color,
    'result': df['result'].map(result_mapping),
    'moves': df['white.moves'],
    'rating': df[rating],
    'rating_difference': (df['black.elo'].astype(int) - df['white.elo'].astype(int)) * flip_sign,
    'distance': df[distance].round(2),
    'distance_difference': (df['black.distance'].astype(float) - df['white.distance'].astype(float)).round(2) * flip_sign,
    'effort': df[effort].round(2),
    'effort_difference': (df['black.perMove'].astype(float) - df['white.perMove'].astype(float)).round(2) * flip_sign,
  })

  return result_df

def merge_sort_by_index(df1, df2):
  return pd.concat([df1, df2]).sort_values(by=['index', 'color'], ascending=[True, False], axis=0)

  
def explode_lines_and_clean(df):
  white_df = split_by_color(df, True)
  black_df = split_by_color(df, False)
  return merge_sort_by_index(white_df, black_df)

In [5]:
# Process and save file

result_df = explode_lines_and_clean(df)
result_df.to_csv('output.csv', index=False)